In [20]:
import requests, json, csv, os
from datetime import datetime

# Endpoint
url = "https://graphql.coincap.io/"

# GraphQL Query
query = """
{
  assets(first: 10000) {
    edges {
      node {
        id
        symbol
        name
        rank
        priceUsd
        changePercent24Hr
        volumeUsd24Hr
        marketCapUsd
        supply
        vwapUsd24Hr
        website
        explorer
        logo
      }
    }
  }
}
"""

def safe_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return None


def fetch_crypto_data():
    """Fetch data from GraphQL API"""
    response = requests.post(url, json={"query": query})
    if response.status_code != 200:
        raise Exception(f"❌ API call failed: {response.status_code}\n{response.text}")
    data = response.json()
    if "data" not in data:
        print("⚠️ No 'data' in response:", data)
        return []
    assets = data["data"]["assets"]["edges"]
    cleaned = []
    for i in assets:
        a = i["node"]
        cleaned.append({
            "timestamp": datetime.utcnow().isoformat(),
            "id": a.get("id"),
            "symbol": a.get("symbol"),
            "name": a.get("name"),
            "rank": a.get("rank"),
            "priceUsd": safe_float(a.get("priceUsd")),
            "changePercent24Hr": safe_float(a.get("changePercent24Hr")),
            "volumeUsd24Hr": safe_float(a.get("volumeUsd24Hr")),
            "marketCapUsd": safe_float(a.get("marketCapUsd")),
            "supply": safe_float(a.get("supply")),
            "vwapUsd24Hr": safe_float(a.get("vwapUsd24Hr")),
            "website": a.get("website"),
            "explorer": a.get("explorer"),
            "logo": a.get("logo")
        })
    return cleaned


def append_to_csv(data, filename="coincap_master_data.csv"):
    """Append new data to a single CSV file (create if not exists)"""
    if not data:
        print("⚠️ No data to append.")
        return

    file_exists = os.path.isfile(filename)
    keys = data[0].keys()

    with open(filename, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        # Write header only once (if file is new)
        if not file_exists:
            writer.writeheader()
        writer.writerows(data)

    print(f"✅ Appended {len(data)} records to {filename} at {datetime.utcnow().strftime('%H:%M:%S UTC')}")


# Run once — fetch and append data
crypto_data = fetch_crypto_data()
append_to_csv(crypto_data)
print(f"✅ Collected {len(crypto_data)} records successfully.")


✅ Appended 1472 records to coincap_master_data.csv at 23:15:55 UTC
✅ Collected 1472 records successfully.
